- [Johansen Cointegration Test: Learn How to Implement it in Python](https://blog.quantinsti.com/johansen-test-cointegration-building-stationary-portfolio/)
- [Unveiling Cointegration: Johansen Test Explained with Python Examples](https://medium.com/@cemalozturk/unveiling-cointegration-johansen-test-explained-with-python-examples-db8385219f1f)
- [Cointegration and Pairs Trading](https://letianzj.github.io/cointegration-pairs-trading.html)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import statsmodels.api as sm

from itertools import combinations, permutations
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from typing import Literal

In [2]:
# Import price data
df = pd.read_csv('prices.txt', engine='python', sep='   ', header=None, names=[f"stock{i}" for i in range(50)])
stocks = df.columns

df_label = 'Price'
if df_label == 'Log Returns':
    # convert data to log returns
    df = np.log1p(df.pct_change())
    df.dropna(axis=0, inplace=True)

df

,stock0,stock1,stock2,stock3,stock4,stock5,stock6,stock7,stock8,stock9,...,stock40,stock41,stock42,stock43,stock44,stock45,stock46,stock47,stock48,stock49
0,13.46,71.65,48.46,50.52,52.10,13.00,18.98,47.71,69.49,49.96,...,32.64,55.76,14.46,58.94,36.71,52.62,49.33,36.22,49.00,56.09
1,13.48,72.10,48.52,50.50,52.06,12.95,18.95,47.84,69.73,49.93,...,32.52,55.97,14.44,59.81,36.64,52.58,49.20,36.27,48.84,56.08
2,13.47,72.35,48.48,50.62,51.80,12.79,18.98,47.98,69.60,49.33,...,32.48,56.34,14.50,59.04,36.89,52.49,49.48,36.39,48.56,55.90
3,13.53,72.51,48.42,50.75,51.66,12.66,18.96,48.74,69.54,49.67,...,32.59,56.32,14.40,58.73,36.94,52.40,49.42,36.41,49.00,56.14
4,13.64,71.99,48.40,50.65,51.97,12.62,18.89,48.88,69.68,49.46,...,32.64,56.32,14.36,59.01,37.03,52.44,49.79,36.42,48.14,55.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,13.69,67.73,46.64,45.72,52.42,10.07,18.05,48.42,69.00,56.19,...,30.73,75.82,10.63,61.35,32.85,50.57,64.36,33.74,33.83,53.47
496,13.58,67.75,46.65,45.71,52.57,10.07,17.99,47.65,69.04,56.00,...,30.70,76.46,10.65,61.20,32.60,50.54,64.65,33.78,33.42,53.91
497,13.69,67.83,46.66,45.66,52.38,10.02,18.02,46.87,68.95,56.09,...,30.70,75.16,10.75,60.82,32.79,50.42,64.28,33.60,33.75,54.22
498,13.55,67.61,46.73,45.62,52.29,10.02,18.03,46.21,69.03,56.51,...,30.69,76.09,10.68,60.73,32.45,50.31,63.60,33.79,33.53,54.50


In [3]:
# Make plot directories
for dirname in ['acf-pacf/', 'cadf/', 'johansen']:
    if not os.path.isdir(dirname):
        os.makedirs(dirname)

#### Autocorrelation + Partial Autocorrelation

In [4]:
def acf_pacf_plot(data, lags=20):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
    sm.graphics.tsa.plot_acf(data.values.squeeze(), ax=ax[0], lags=lags)
    sm.graphics.tsa.plot_pacf(data.values.squeeze(), ax=ax[1], lags=lags)
    fig.savefig(f"acf-pacf/{data.name}.png")
    plt.close(fig)

# Plot ACF/PACF of all stocks
for stock in stocks:
    acf_pacf_plot(df[stock])

#### Cointegration Augmented Dicky-Fuller Test

In [5]:
def hedge_ratio(Y, X, eval: Literal['spread', 'residuals']):
    model = sm.OLS(Y, X)
    model = model.fit()

    if eval == 'spread':
        constant = model.params.iloc[0]
        return Y - constant * X
    else:
        return model.resid

def cadf_test(Y, X, maxlag=1, significance=0.1):
    # Hedge ratio
    hedge = hedge_ratio(Y, X, 'residuals')

    # Compute ADF test statistics
    adf = adfuller(hedge, maxlag)
    t_stat = adf[0]
    p_val = adf[1]
    crit_90 = adf[4]['10%']
    crit_95 = adf[4]['5%']
    crit_99 = adf[4]['1%']

    # Check co-integration
    if t_stat <= crit_90 and p_val <= significance:
        confidence = 90
    elif t_stat <= crit_95 and p_val <= significance:
        confidence = 95
    elif t_stat <= crit_99 and p_val <= significance:
        confidence = 99
    else:
        confidence = None

    return hedge, confidence, t_stat

def plot_cadf_test(hedge, Y, X, confidence):
    y_str, x_str = Y.name, X.name

    # Save spread plot for reference
    title = f"Cointegration between y={y_str} x={x_str}:\n{confidence}% confidence"

    fig, (ax1, ax2) = plt.subplots(1, 2)
    # plot spread
    ax1.plot(hedge)
    ax1.set_ylabel('Hedge Ratio')

    # plot historical prices
    ax2.plot(Y, label=y_str)
    ax2.plot(X, label=x_str)

    ax2.set_ylabel(df_label)
    ax2.yaxis.set_label_position('right')
    ax2.yaxis.tick_right()

    ax2.legend(loc='lower left')

    plt.suptitle(title)
    plt.savefig(f"cadf/{y_str}-{x_str}")
    plt.close(fig)

# Conduct ADF test for all stocks
confidence_threshold = 95
sig_threshold = 0.1

# for s1, s2 in [('stock0', 'stock30')]:        # single pair
for s1, s2 in combinations(stocks, 2):
    hedge1, confidence1, t_stat1 = cadf_test(df[s1], df[s2], significance=sig_threshold)
    hedge2, confidence2, t_stat2 = cadf_test(df[s2], df[s1], significance=sig_threshold)

    # Choose the t-statistic with most negative / statistically significant test-statistic
    if not confidence1 or not confidence2:
        continue
    elif t_stat1 < t_stat2 and confidence1 >= confidence_threshold:
        plot_cadf_test(hedge1, df[s1], df[s2], confidence1)
    elif t_stat2 < t_stat1 and confidence2 >= confidence_threshold:
        plot_cadf_test(hedge2, df[s2], df[s1], confidence2)

#### Johansen Cointegration Test

In [11]:
def johansen_test(data, det_order=0, k_ar_diff=1, debug=False):
    """Performs the Johansen cointegration test and prints the results.

    Args:
        data (np.ndarray): Time series data for cointegration test
        det_order (int): The order of the deterministic terms. Defaults to 0.
                        -1: (most restrictive) No constant or trend
                        0: Constant term only
                        -1: (least restrictive) Constant and trend terms
        k_ar_diff (int): The number of lags to include in the VAR model. Defaults to 1.
    """

    # Compute Johansen test statistics
    jh_results = coint_johansen(data, det_order, k_ar_diff)

    trace_stat = jh_results.lr1
    crit_val_table = jh_results.cvt
    eigenvectors = jh_results.evec

    if debug:
        print(trace_stat)   # dim=(n,) | Trace statistic
        print(crit_val_table)   # dim=(n,3) | Critical value table (90%, 95%, 99%)
        print(eigenvectors)  # dim=(n,n) | column-wise eigenvectors

    # Check co-integration
    confidence, rank = None, None
    confidence_levels = [90, 95, 99]

    for i, trace in enumerate(trace_stat):
        for j, conf_level in enumerate(crit_val_table[i]):
            if trace > conf_level:
                confidence = confidence_levels[j]
                rank = i

    return confidence, rank

def plot_johansen_test(data, confidence, rank):
    # Save spread plot for reference
    title = f"Cointegration between {data.columns}:\n r={rank} up to {confidence}% confidence"

    fig = plt.figure()

    # plot historical prices
    plt.plot(data, label=data.columns)
    plt.ylabel(df_label)
    plt.legend()
    plt.suptitle(title)

    plt.savefig(f"johansen/{data.columns[0]}-{data.columns[1]}")
    plt.close(fig)

# Conduct Johansen test for all stocks
confidence_threshold = 99
rank_threshold = 1

# for s1, s2 in [('stock18', 'stock43')]:        # single pair
for s1, s2 in combinations(stocks, 2):
    confidence, rank = johansen_test(df[[s1, s2]], debug=True)

    if confidence and rank and confidence >= confidence_threshold and rank >= rank_threshold:
        plot_johansen_test(df[[s1, s2]], confidence, rank)

[12.89400418  3.11530374]
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
[[ 1.29539109 -0.82930616]
 [-0.25995206 -0.13855206]]
[6.48186417 1.66534128]
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
[[ 0.06082285 -1.33598511]
 [-1.63292368 -0.07665799]]
[7.37075063 1.59044605]
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
[[ 1.4102105   0.22585011]
 [-0.2538898   0.45443817]]
[10.33179896  2.77935155]
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
[[ 0.98604836 -1.01416447]
 [-0.55549859 -0.2656882 ]]
[9.72683793 2.01134723]
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
[[ 1.3980194   0.22606195]
 [-0.77355614  1.49373766]]
[8.52711032 1.29509086]
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
[[ 0.31407584 -1.30316844]
 [-3.31891452 -0.56200403]]
[16.0222565   1.70375576]
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
[[ 0.2733284  -1.327169  ]
 [-0.80872266 -0.03400752]]
[11.6106648   1.80618739]
[[13.4294 15.4943 19.9349]
 [ 

### Testing the exact number of cointegrating relationships

In [ ]:


# Interpret the results for each pair
for i, (stock1, stock2) in enumerate(combinations(rand_stocks, 2)):
    if (tracevalues[i] > critical_values[:, 1]).all():
        print(f"Pair {i + 1} ({stock1} and {stock2}) is cointegrated.")
    else:
        print(f"Pair {i + 1} ({stock1} and {stock2}) is not cointegrated.")